# The Harmony MaskFill service

The Harmony MaskFill service takes gridded cloud-hosted input Earth Observation (EO) data granules, alongside a GeoJSON shape file, and masks all pixels outside the requested GeoJSON shape. These pixels are set to the fill value associated with each variable in question.

This Jupyter notebook will demonstrate how data can be retrieved from the Harmony MaskFill service using the NASA `harmony-py` Python package.

Note, that the MaskFill service is currently not used in isolation, but as part of a service chain, calling the Harmony OPeNDAP SubSetter (HOSS) first to crop the output to a minimally encompassing bounding box, before using MaskFill to mask all pixels inside that box, but outside the specified GeoJSON shape file.

### Environment setup:

This notebook assumes that it is being run in a local Python environment, configured using either `pyenv` or conda. Either can be used, but the dependencies will be installed via Pip. To install the required packages to run this notebook:

```bash
$ pip install -r requirements.txt
```

Note - your environment will require the [geos](https://trac.osgeo.org/geos/) package to be able to install all the dependencies for this notebook (particularly `cartopy`).

`harmony-py` is available from [PyPI](https://pypi.org/project/harmony-py/). It can also be installed with Pip:

```bash
$ pip install harmony-py
```

### Notebook setup:

`harmony-py` contains functionality to authenticate with Earthdata Login. It requires a `.netrc` (`_netrc` on Windows) file in your home directory:

```
machine uat.urs.earthdata.nasa.gov
    login narmstrong
    password ap0ll0
```

Make sure that this file is only readable by the current user or you will receive an error stating "netrc access too permissive."

```
$ chmod 0600 ~/.netrc
```

Note that the Earthdata Login environment must match the Harmony environment containing the collection and service later in the notebook (e.g. UAT or production).

#### Plotting functions

The function below can be used to plot information from either the granule itself, or regarding the shape file.

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import contextily as ctx
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


def plot_variable(variable_data, longitudes, latitudes, fill_value=None, title=None,
                  colourbar_units=None):
    """ This helper function will create a masked array, to remove fill values, then
        it will load features using `cartopy` to display oceans, land, coastlines and
        rivers on the output image. Then it will display a contour plot of the data,
        before
    
        Plots variable against longitudes and latitudes in masked area on the world map.

    """
    # Mask out fill values:
    masked_variable = np.ma.masked_where(variable_data[:] == fill_value, variable_data)

    fig = plt.figure(figsize=(10, 10))
    if title is not None:
        fig.suptitle(title, fontsize=20)

    # Include basic features in contour plot:
    ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=0))
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.RIVERS)

    # Plot masked data:
    colour_scale = ax.contourf(longitudes, latitudes, masked_variable, levels=20)
    
    # Add colour bar for scaling
    colour_bar = plt.colorbar(colour_scale, ax=ax, orientation='horizontal', pad=0.05)
    if colourbar_units is not None:
        colour_bar.set_label(colourbar_units, fontsize=14)

    plt.tight_layout()
    plt.show()


def plot_shape_file(shape_file_path):
    """ Plots a shape file from a GeoJSON input file against a background image
        showing a map of the Earth.

    """
    shape = gpd.read_file(shape_file_path).to_crs(epsg=3857)
    plot = shape.plot(alpha=0.5, edgecolor='k', figsize=(8, 8))
    ctx.add_basemap(plot)

# Information required by the Harmony request:

The Harmony shape file subsetting request require a few pieces of information:

* The concept ID of the data collection containing the granules to be processed. This has the format "C1234567890-[PROVIDER]".
* The concept ID of the granule(s) being requested. This has the format "G1234567890-[PROVIDER]".
* A shape file, saved as a GeoJSON file. (See below for more details on shape files).
* Harmony endpoint URL.

### Shape files:

When making requests directly to Harmony, these shape files should contain one or more `Polygon` features. `MultiPolygon` features will not be accepted by the API, and should be split into a list of separate `Polygon` features. In addition all polygons should have their points ordered in an anticlockwise direction.

## The test case:

This test request will retrieve a granule from the [SMAP level 4, Net Carbon Ecosystem Exchange collection](https://nsidc.org/data/spl4cmdl) (SPL4CMDL). The SPL4CMDL collections shows estimates of Carbon Dioxide exchange, on the EASE-2 global (cylindrical) grid.

The region of interest, as specified in the shape file, will be the Amazon river basin.

In [ ]:
plot_shape_file('amazon_basin.geo.json')

# Using `harmony-py` to make MaskFill requests

We will use the SPL4CMDL collection and a sample granule, alongside the Amazon river basin GeoJSON shape, and make a Harmony request using the `harmony-py` package.

As noted above, the `harmony-py` package can be installed from the Python Package Index using Pip:

```bash
$ pip install -U harmony-py
```

`harmony-py` creates a client, which requires Earthdata Login credientials. If a `.netrc` file exists in the home directory of your machine, the `harmony-py` client will attempt to retrieve Earthdata Login credentials from it.

### The `harmony-py` `Client`:

This class will handle the communication with the Harmony API, including authentication, as derived from a `.netrc` file. Requests are made by submitting a `Request` instance to Harmony using the `Client` instance. The `harmony-py` `Client` can be configured for any of the available Harmony environments, e.g. UAT or production.

### Forming a `Request` instance:

An instance of the `Request` object can be used to make requests to the Harmony API via the a harmony `Client` instance. To create a `Request` object, the collection must be specified. Additional arguments can be included. For example the concept IDs of granules or variables to be processed. Omitting granule IDs will result in Harmony trying to process all granules in a collection. Similarly, not specifying variables is the equivalent of requesting all variables in each granule.

In [ ]:
from harmony import Client, Collection, Request, Environment


# Define the collection and granule concept IDs:
collection_id = 'C1240150677-EEDTEST'
granule_id = 'G1245558670-EEDTEST'

# Create a client for the UAT environment:
harmony_client = Client(env=Environment.UAT)

# Specify the path to the locally saved GeoJSON shape file:
shapefile = 'amazon_basin.geo.json'

# Create a collection instance, using the concept ID:
collection = Collection(id=collection_id)

# Construct a request, ready to send to the Harmony API:
request = Request(collection=collection, granule_id=[granule_id], shape='amazon_basin.geo.json')

# Ensure the request is valid:
print(f'Request is valid: {request.is_valid()}')

## Submit requests and tracking progress

Submitting a valid request to Harmony using the `Client` instance will return a job ID. This ID can be used to track the job status using in-built functionality.

In [ ]:
# Submit a valid Harmony request via the UAT Harmony client.
job_id = harmony_client.submit(request)

# Show job progress
print(f'\nWaiting for the job {job_id} to finish')
results = harmony_client.result_json(job_id, show_progress=True)

### Downloading data:

The following cell will download all the processed output. The `Client.download_all` method will save local copies of each granule, and will return information about those files.

In [ ]:
print('\nDownloading results.')
futures = harmony_client.download_all(job_id, overwrite=True)
downloaded_files = [future.result() for future in futures]
print(f'Downloaded files: {downloaded_files}')

### Response JSON:

If we're just interested in the json Harmony produces we can retrieve that also. This could be a useful way to retrieve the URLs of the processed data, without downloading those files until they are actually needed. It also provided information on the STAC catalogs for the output granules.

In [ ]:
harmony_client.result_json(job_id)

### Assessing the output:

The cell below will plot the environmental constraint multiplier from the downloaded file retrieved by `harmony-py`. It should be identical to the output from using the Harmony API directly.

In [ ]:
from h5py import File as H5File


# Open the first output file:
harmony_py_data = H5File(downloaded_files[0], 'r')

# Plot the mean global net ecosystem exchange:
plot_variable(harmony_py_data['/NEE/nee_mean'], harmony_py_data['/GEO/longitude'], harmony_py_data['/GEO/latitude'],
              fill_value=-9999.0, title='Mean global daily 9km Net Ecosystem Exchange',
              colourbar_units=r'$\mathregular{g.cm^{-2}.day^{-1}}$')

## NOTE: The rest of this notebook requires you to be running it in us-west-2.

To do so follow tutorial 3 of the [NASA Earthdata Cloud Primer](https://earthdata.nasa.gov/learn/user-resources/webinars-and-tutorials/cloud-primer), but also open an inbound port (port 8888) in the security group settings of the EC2 instance, as detailed [here](https://medium.com/@alexjsanchez/python-3-notebooks-on-aws-ec2-in-15-mostly-easy-steps-2ec5e662c6c6). For more detailed instructions regarding running a Harmony Jupyter notebook in an EC2 instance, see [here](https://github.com/nasa/harmony/blob/main/docs/Harmony_20.3Demo.ipynb).

### STAC catalog output

Each successful Harmony request will return a [STAC](https://stacspec.org/) catalog. STAC allows for scripts to access information regarding assets, without first having to download and consume the full asset. With granules that can be hundreds of megabytes (or even several gigabytes) in size, this allows a user to inspect or even filter granules and optimise their workflow by only obtaining the data that are most useful to their analyses.

The STAC catalog also includes the S3 location of each output file allowing for direct S3 access. Such information can be used by other cloud services, enabling in-place data analysis.

In [ ]:
from pystac import Catalog

stac_catalog_url = harmony_client.stac_catalog_url(job_id)
catalog = Catalog.from_file(stac_catalog_url)

stac_s3_links = [asset.href for item in catalog.get_all_items() for asset in item.assets.values()]
print(stac_s3_links)

The metadata for each STAC item can be displayed to see information such as the bounding box, coordinates and start time or end time of the item.

In [ ]:
import intake

intake_catalog = intake.open_stac_catalog(stac_catalog_url, name='Harmony MaskFill')

for stac_id, stac_item in intake_catalog.items():
    print(f'{stac_id}\n{stac_item.metadata}\n\n')

### Cloud in-place analysis:

Instead of requesting HTTPS URLs, which are the default, it is possible to request S3 URLs. These can be used to perform analysis in-place, or to retrieve the data directly from S3.

In [ ]:
from harmony import LinkType

s3_output_urls = harmony_client.result_urls(job_id, link_type=LinkType.s3)
s3_output_urls

#### Accessing the data in-place:

With the release of version 3.2, `h5py` is now able to use the ROS3 driver to read data from an object in S3. This does require some specific configuration of HDF5 and installation of `h5py`, as detailed [here](https://docs.h5py.org/en/stable/whatsnew/3.2.html?highlight=ros3#new-features). The next cell will generate some temporary AWS credential to enable access to the S3 bucket containing the results of the Harmony request:

In [ ]:
aws_credentials = harmony_client.aws_credentials()

#### Plotting data directly from S3:

The cell below shows how `h5py`, if configured with the ROS3 driver can allow for plotting of data as hosted in S3:

In [ ]:
#
# This cell will _only_ work if you follow the installation instructions here:
# https://docs.h5py.org/en/stable/whatsnew/3.2.html?highlight=ros3#new-features
# The ROS3 driver does not come automatically included in h5py == 3.2 as available
# from PyPI.
#
from h5py import File as H5File


# Open the output file stored in S3:
s3_output_file = H5File(s3_output_urls[0], 'r', driver='ros3', aws_region='us-west-2',
                        secret_id=aws_credentials['aws_access_key_id'],
                        secret_key=aws_credentials['aws_secret_access_key'])

# Plot the mean global net ecosystem exchange:
plot_variable(s3_output_file['/NEE/nee_mean'], s3_output_file['/GEO/longitude'],
              s3_output_file['/GEO/latitude'], fill_value=-9999.0,
              title='Mean global daily 9km Net Ecosystem Exchange',
              colourbar_units=r'$\mathregular{g.cm^{-2}.day^{-1}}$')

#### Downloading from S3:

The following cell shows how the output files can be downloaded directly from S3:

In [ ]:
from harmony import s3_components
import boto3

s3 = boto3.client('s3', **aws_credentials)

for s3_output_url in s3_output_urls:
    s3_bucket, s3_object, file_name = s3_components(s3_output_url)

    with open(file_name, 'wb') as file_handler:
        s3.download_fileobj(s3_bucket, s3_object, file_handler)